In [2]:
# =========================
# Project Two – Dashboard
# =========================

# Dash (Jupyter) setup
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
JupyterDash.infer_jupyter_proxy_config()

# Visuals
import dash_leaflet as dl
import plotly.express as px

# Data / utils
import base64
import pandas as pd
import numpy as np

# Import your CRUD module (Project One)
from CRUD_Python_Module import AnimalShelter   # <— IMPORTANT

# ---------------------------
# Model (Mongo via CRUD)
# ---------------------------
USERNAME = "aacuser"
PASSWORD = "D@rkLumoo28"

db = AnimalShelter(user=USERNAME, passwd=PASSWORD)

def read_all_df():
    """Read all records from Mongo and return a cleaned DataFrame."""
    data = db.read({}, projection=None)
    df = pd.DataFrame.from_records(data)
    if "_id" in df.columns:
        df.drop(columns=["_id"], inplace=True)
    # Make sure expected numeric columns are numeric if present
    for c in ["age_upon_outcome_in_weeks", "location_lat", "location_long"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

# Initial unfiltered data
df = read_all_df()

# ---------------------------
# Branding: Logo + Identifier
# ---------------------------
# Put the logo file in code_files and reference it here:
LOGO_FILE = "Grazioso Salvare Logo.png"  # exact name + path

try:
    encoded_image = base64.b64encode(open(LOGO_FILE, "rb").read()).decode()
    logo_img = html.A(
        href="https://www.snhu.edu",
        target="_blank",
        children=html.Img(
            src=f"data:image/png;base64,{encoded_image}",
            style={"height": "60px"}
        )
    )
except Exception:
    logo_img = html.Div("Logo not found", style={"color": "#999"})

# ---------------------------
# Controller: filter → query
# ---------------------------
def build_query(filter_value: str) -> dict:
    """
    Map radio selection to Mongo query per specs.
    Uses age_upon_outcome_in_weeks, sex_upon_outcome, breed, animal_type.
    """
    # Common constraint: we're only interested in dogs for these rescues
    base = {"animal_type": "Dog"}

    if filter_value == "Water Rescue":
        # Breeds + intact female + 26–156 weeks
        breed_or = {
            "$or": [
                {"breed": {"$regex": "Labrador Retriever", "$options": "i"}},
                {"breed": {"$regex": "Chesapeake Bay Retriever", "$options": "i"}},
                {"breed": {"$regex": "Newfoundland", "$options": "i"}},
            ]
        }
        age = {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
        sex = {"sex_upon_outcome": "Intact Female"}
        return {**base, **breed_or, **age, **sex}

    elif filter_value == "Mountain/Wilderness Rescue":
        breed_or = {
            "$or": [
                {"breed": {"$regex": "German Shepherd", "$options": "i"}},
                {"breed": {"$regex": "Alaskan Malamute", "$options": "i"}},
                {"breed": {"$regex": "Old English Sheepdog", "$options": "i"}},
                {"breed": {"$regex": "Siberian Husky", "$options": "i"}},
                {"breed": {"$regex": "Rottweiler", "$options": "i"}},
            ]
        }
        age = {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
        sex = {"sex_upon_outcome": "Intact Male"}
        return {**base, **breed_or, **age, **sex}

    elif filter_value == "Disaster/Individual Tracking":
        breed_or = {
            "$or": [
                {"breed": {"$regex": "Doberman Pinscher", "$options": "i"}},
                {"breed": {"$regex": "German Shepherd", "$options": "i"}},
                {"breed": {"$regex": "Golden Retriever", "$options": "i"}},
                {"breed": {"$regex": "Bloodhound", "$options": "i"}},
                {"breed": {"$regex": "Rottweiler", "$options": "i"}},
            ]
        }
        age = {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
        sex = {"sex_upon_outcome": "Intact Male"}
        return {**base, **breed_or, **age, **sex}

    # Reset → no filter (all dogs)
    return {}

# ---------------------------
# View (Dash layout)
# ---------------------------
app = JupyterDash("Grazioso_Salvare_Dashboard")

app.layout = html.Div([
    # Top bar: logo + name
    html.Div(
        style={"display": "flex", "alignItems": "center", "gap": "12px"},
        children=[
            logo_img,
            html.H2("Grazioso Salvare – Ivan Gonzalez", style={"margin": 0}),
        ],
    ),
    html.Div("Project Two: Interactive Dashboard (Filters → Table → Map & Chart)", style={"color": "#666"}),
    html.Hr(),

    # Filters
    html.Div([
        html.Label("Rescue Type Filter:"),
        dcc.RadioItems(
            id="filter-type",
            options=[
                {"label": "Water Rescue", "value": "Water Rescue"},
                {"label": "Mountain/Wilderness Rescue", "value": "Mountain/Wilderness Rescue"},
                {"label": "Disaster/Individual Tracking", "value": "Disaster/Individual Tracking"},
                {"label": "Reset (All Dogs)", "value": "Reset"},
            ],
            value="Reset",  # default
            labelStyle={"display": "inline-block", "marginRight": "18px"}
        ),
    ], style={"marginBottom": "10px"}),

    html.Hr(),

    # Data table
    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name": c, "id": c, "deletable": False, "selectable": True} for c in df.columns],
        data=df.to_dict("records"),

        # Good UX options
        page_size=15,
        sort_action="native",
        filter_action="native",
        column_selectable="single",
        row_selectable="single",
        selected_rows=[0],  # select the first row by default so the map has data
        style_table={"overflowX": "auto", "maxHeight": "420px", "overflowY": "auto"},
        style_cell={
            "textAlign": "left",
            "padding": "6px",
            "minWidth": "100px", "width": "120px", "maxWidth": "300px",
            "whiteSpace": "normal",
        },
        style_header={"backgroundColor": "#f1f1f1", "fontWeight": "bold"},
        style_data_conditional=[
            {"if": {"row_index": "odd"}, "backgroundColor": "#fafafa"}
        ],
        export_format="csv",
    ),

    html.Br(),
    html.Hr(),

    # Two visuals side-by-side
    html.Div(
        className="row",
        style={"display": "flex", "gap": "16px"},
        children=[
            html.Div(id="graph-id", className="col s12 m6"),
            html.Div(id="map-id", className="col s12 m6"),
        ],
    ),
])

# -----------------------------------------------
# Callbacks
# -----------------------------------------------

# 1) Filter → update table data
@app.callback(
    Output("datatable-id", "data"),
    [Input("filter-type", "value")]
)
def update_table(filter_value):
    query = build_query(filter_value)
    data = db.read(query, projection=None)
    if not data:
        return []
    dff = pd.DataFrame.from_records(data)
    if "_id" in dff.columns:
        dff.drop(columns=["_id"], inplace=True)
    # some columns may need numeric coercion again (if filtered)
    for c in ["age_upon_outcome_in_weeks", "location_lat", "location_long"]:
        if c in dff.columns:
            dff[c] = pd.to_numeric(dff[c], errors="coerce")
    return dff.to_dict("records")

# 2) Highlight selected column for readability
@app.callback(
    Output("datatable-id", "style_data_conditional"),
    [Input("datatable-id", "selected_columns")]
)
def highlight_cols(selected_columns):
    return [{
        "if": {"column_id": c},
        "background_color": "#D2F3FF"
    } for c in (selected_columns or [])]

# 3) Table (filtered) → Bar chart of breed counts
@app.callback(
    Output("graph-id", "children"),
    [Input("datatable-id", "derived_virtual_data")]
)
def update_breed_chart(viewData):
    if not viewData:
        return [html.Div("No data to chart.")]
    dff = pd.DataFrame.from_dict(viewData)
    if "breed" not in dff.columns:
        return [html.Div("Breed column not found.")]
    counts = dff["breed"].value_counts().reset_index()
    counts.columns = ["breed", "count"]
    # Limit to top 12 for readability
    counts = counts.head(12)
    fig = px.bar(
        counts,
        x="breed",
        y="count",
        title="Breed Counts (Top 12)",
        labels={"breed": "Breed", "count": "Count"},
    )
    fig.update_layout(margin=dict(l=10, r=10, t=40, b=10), height=420)
    return [dcc.Graph(figure=fig)]

# 4) Table (filtered + selected row) → Leaflet map
@app.callback(
    Output("map-id", "children"),
    [
        Input("datatable-id", "derived_virtual_data"),
        Input("datatable-id", "derived_virtual_selected_rows"),
    ]
)
def update_map(viewData, selected_rows):
    if not viewData:
        return [html.Div("No data to map.")]
    dff = pd.DataFrame.from_dict(viewData)

    row = 0
    if selected_rows and len(selected_rows) > 0:
        row = selected_rows[0]

    # Column names expected from AAC dataset
    lat_col = "location_lat"
    lon_col = "location_long"
    breed_col = "breed" if "breed" in dff.columns else None
    name_col = "name" if "name" in dff.columns else None

    # If coordinates are missing/invalid, show friendly message
    try:
        lat = float(dff.iloc[row][lat_col])
        lon = float(dff.iloc[row][lon_col])
        if np.isnan(lat) or np.isnan(lon):
            raise ValueError("Missing coordinates")
    except Exception:
        return [html.Div("Selected row has no valid coordinates to plot.")]

    tooltip_text = dff.iloc[row][breed_col] if breed_col else "Animal"
    popup_name  = dff.iloc[row][name_col] if name_col else "(no name)"

    center = [30.75, -97.48]  # Austin approx

    return [
        dl.Map(
            style={"width": "100%", "height": "420px"},
            center=center, zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(tooltip_text),
                        dl.Popup([html.H3("Animal Name"), html.P(popup_name)])
                    ]
                )
            ]
        )
    ]

# Run app. If 8050 is busy, change the port.
app.run_server(host="0.0.0.0", port=8050, debug=False)

 * Running on all addresses.
 * Running on http://10.122.188.59:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Oct/2025 20:57:43] "GET /_alive_9397d07a-7b18-4a6d-8330-15492ef8c98b HTTP/1.1" 200 -


Dash app running on https://capitalplato-ultrasport-3000.codio.io/proxy/8050/


127.0.0.1 - - [11/Oct/2025 21:02:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2025 21:02:15] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_8_0m1752168217.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2025 21:02:15] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_8_1m1752168217.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2025 21:02:16] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_8_0m1752168217.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2025 21:02:16] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_8_1m1752168217.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2025 21:02:16] "GET /_dash-component-suites/dash/deps/react@16.v2_8_1m1752168217.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2025 21:02:16] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_8_1m1752168217.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2025 21:02:16] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_8_1m1752168217.8.1.min.js HTT